In [38]:
import tqdm
import pandas as pd
from pathlib import Path
import xarray as xr

In [41]:
p = Path('/mnt/team/rapidresponse/pub/population/modeling/climate_malnutrition/zzz_archive/v1/results/tempgrid_o30/')

def get_forecasted_population(location_id, year, sex_id, age_group_id):
    FORECASTED_POPULATIONS_FILEPATH = '/mnt/share/forecasting/data/7/future/population/20240529_500d_2100_lhc_ref_squeeze_hiv_shocks_covid_all_gbd_7_shifted/population.nc'
    future_pop_x = xr.open_dataset(FORECASTED_POPULATIONS_FILEPATH)
    filtered_pop_x = future_pop_x.loc[dict(year_id=year, location_id = location_id, sex_id = sex_id, age_group_id=age_group_id)]
    mean_population = filtered_pop_x.mean(dim='draw').population.to_numpy()[0]
    return mean_population

def get_forecasted_population_full():
    FORECASTED_POPULATIONS_FILEPATH = '/mnt/share/forecasting/data/7/future/population/20240529_500d_2100_lhc_ref_squeeze_hiv_shocks_covid_all_gbd_7_shifted/population.nc'
    future_pop_ds = xr.open_dataset(FORECASTED_POPULATIONS_FILEPATH)
    mean_population = future_pop_ds.mean(dim='draw').population.to_numpy()[0]
    return mean_population

In [15]:
dfs = []
for fp in tqdm.tqdm(list(p.glob("stunting_*_ssp245_*"))):
    df = pd.read_parquet(fp)
    dfs.append(df)

100%|██████████████████████████████████████████████████████████| 42679/42679 [17:31<00:00, 40.57it/s]


In [16]:
pd.concat(dfs).to_parquet('ssp245.parquet')

In [17]:
ssp119 = pd.read_parquet('ssp119.parquet')
ssp245 = pd.read_parquet('ssp245.parquet')

In [65]:
combined = pd.concat([
    ssp245.set_index('year_id', append=True).affected_proportion.rename('ssp245'),
    ssp119.set_index('year_id', append=True).affected_proportion.rename('ssp119'),
], axis=1).sort_index()
combined = combined.reset_index().rename(columns={'fhs_location_id': 'location_id'}).assign(location_id = lambda x: x.location_id.astype(int))
combined = combined[combined.year_id > 2020]
combined = combined.set_index(['location_id', 'age_group_id', 'sex_id', 'year_id']).sort_index()

In [66]:
combined

ssp245    ssp119
location_id age_group_id sex_id year_id                    
7           4            1      2021     0.372456  0.372456
                                2022     0.372984  0.372984
                                2023     0.373290  0.373290
                                2024     0.373542  0.373542
                                2025     0.373580  0.373580
...                                           ...       ...
60137       5            2      2096     0.373940  0.373940
                                2097     0.373907  0.373907
                                2098     0.373875  0.373875
                                2099     0.373842  0.373842
                                2100     0.373810  0.373810

[150080 rows x 2 columns]

In [57]:
FORECASTED_POPULATIONS_FILEPATH = '/mnt/share/forecasting/data/7/future/population/20240529_500d_2100_lhc_ref_squeeze_hiv_shocks_covid_all_gbd_7_shifted/population.nc'
pop = xr.open_dataset(FORECASTED_POPULATIONS_FILEPATH).mean(dim='draw').sel(age_group_id=[4,5]).to_dataframe().reset_index("scenario", drop=True).reset_index().set_index(['location_id', 'age_group_id', 'sex_id', 'year_id'])

In [70]:
df = pd.concat([combined, pop.loc[combined.index]], axis=1)

In [74]:
df['ssp245_count'] = df['ssp245'] * df['population']
df['ssp119_count'] = df['ssp119'] * df['population']
df['delta'] = df['ssp245_count'] - df['ssp119_count']

In [75]:
df.groupby(['age_group_id', 'sex_id']).delta.sum()

age_group_id  sex_id
4             1        -1.065470e+07
              2        -8.856631e+06
5             1        -8.978653e+07
              2        -7.994703e+07
Name: delta, dtype: float64